In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

from context import Context
from utils.utils_chain import Account, Address
from contracts.simple_lock_contract import SimpleLockContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from contracts.dex_proxy_contract import DexProxyContract
context = Context()
IMPERSONATE_ADDRESS = ""
SIMPLE_LOCK_ENERGY_ADDRESS = ""
LOCKED_TOKEN = "MEX-455c57"
LOCKED_LP_TOKEN = "XMEX-fda355"
LOCKED_FARM_TOKEN = "LKMEX-aab910"

user = Account(pem_file=config.DEFAULT_ACCOUNTS)
# contract = SimpleLockEnergyContract(LOCKED_TOKEN, LOCKED_LP_TOKEN, LOCKED_FARM_TOKEN, SIMPLE_LOCK_ENERGY_ADDRESS)
if "shadowfork" in context.network_provider.proxy.url and IMPERSONATE_ADDRESS:
    user.address = Address.from_bech32(IMPERSONATE_ADDRESS)
    user.sync_nonce(context.network_provider.proxy)


proxy_contract: DexProxyContract
proxy_contract = context.get_contracts(config.PROXIES_V2)[0]
energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

In [ ]:
from multiversx_sdk import Address, TokenPayment, ProxyNetworkProvider
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ESDTNFTTransferBuilder, DefaultTransactionBuildersConfiguration
from tools.notebooks import env

config = DefaultTransactionBuildersConfiguration(chain_id=env.CHAIN_ID)
proxy = ProxyNetworkProvider(env.PROXY_URL)

migrateOldTokens

In [ ]:
from multiversx_sdk import TransactionsConverter

user = env.USER1
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(LKMEX, 2286831, 100000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="migrateOldTokens",
    caller=user,
    call_arguments=[],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()

converter = TransactionsConverter()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
from multiversx_sdk import TransactionComputer()

tx.nonce = user_nonce_holder.get_nonce_then_increment()

tx_computer = TransactionComputer()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)

In [ ]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from utils.utils_chain import Account, WrapperAddress
from utils.utils_tx import ESDTToken
from multiversx_sdk import ProxyNetworkProvider

energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

user = Account(pem_file=config.DEFAULT_OWNER)
user.address = WrapperAddress("erd14sp4unwsgyae0hae8cgnm0tp6yaj3emyt02mxlhfsr48ymqz52lqheyh9s")
user.sync_nonce(context.network_provider.proxy)

proxy2 = ProxyNetworkProvider("https://proxy-shadowfork-one.elrond.ro")

hash = energy_contract.lock_tokens(user, context.network_provider.proxy, [[ESDTToken("MEX-455c57", 0, 10000)], 720])
user.sync_nonce(proxy2)
hash2 = energy_contract.lock_tokens(user, proxy2, [[ESDTToken("MEX-455c57", 0, 10000)], 720])

LOCK TOKENS

In [ ]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from utils.utils_chain import get_token_details_for_address
from utils.utils_tx import ESDTToken

_, mex_amount, _ = get_token_details_for_address(proxy_contract.token, user.address.bech32(), context.network_provider.proxy)

lockable_tokens = ESDTToken(proxy_contract.token, 0, mex_amount)

txhash = energy_contract.lock_tokens(user, context.network_provider.proxy, [[lockable_tokens], 720])

UNLOCK TOKENS

In [ ]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from utils.utils_chain import get_token_details_for_address
from utils.utils_tx import ESDTToken

txhash = energy_contract.unlock_tokens(user, context.network_provider.proxy, [[ESDTToken("XMEX-fda355", 62, 10)]])

EXTEND LOCK

In [ ]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from utils.utils_tx import ESDTToken

# contract = SimpleLockEnergyContract(LOCKED_TOKEN, LOCKED_TOKEN, LOCKED_LP_TOKEN, LOCKED_FARM_TOKEN, SIMPLE_LOCK_ENERGY_ADDRESS)
tx = energy_contract.extend_lock(user, context.network_provider.proxy, [[ESDTToken("XMEX-fda355", 61, 10)], 30])

print("Transaction hash:", tx)

In [ ]:

import random
import time
from ported_arrows.stress.shared import get_shard_of_address
from utils.contract_data_fetchers import SimpleLockEnergyContractDataFetcher
from utils.utils_chain import nominated_amount


deployer_shard = get_shard_of_address(context.deployer_account.address)
# contract: SimpleLockEnergyContract
contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]


simple_lock_energy_data_fetcher = SimpleLockEnergyContractDataFetcher(Address.new_from_bech32(contract.address),
                                                                          context.network_provider.proxy.url)
lock_options = simple_lock_energy_data_fetcher.get_data('getLockOptions')
sleep_time = config.CROSS_SHARD_DELAY if get_shard_of_address(user.address) is not deployer_shard \
        else 6

print(lock_options)
    # lock tokens
amount = random.randint(1, 10)
lock_period = random.choice(lock_options)
token = ESDTToken(LOCKED_TOKEN, 0, nominated_amount(amount))
args = [[token], lock_period]
_ = contract.lock_tokens(user, context.network_provider.proxy, args)

print(f"User: {user.address.bech32()}")
print(f"Locked {amount} tokens for {lock_period} epochs.")
time.sleep(sleep_time)

GET ENERGY FOR USER

In [ ]:
from utils.contract_data_fetchers import SimpleLockEnergyContractDataFetcher


simple_lock_energy_data_fetcher = SimpleLockEnergyContractDataFetcher(Address.new_from_bech32(energy_contract.address),
                                                                          context.network_provider.proxy.url)

user_energy = simple_lock_energy_data_fetcher.get_data('getEnergyEntryForUser', [user.address])
print(user_energy)

In [ ]:
from utils.contract_data_fetchers import SimpleLockEnergyContractDataFetcher


simple_lock_energy_data_fetcher = SimpleLockEnergyContractDataFetcher(Address.new_from_bech32(energy_contract.address),
                                                                          context.network_provider.proxy.url)

user_energy = simple_lock_energy_data_fetcher.get_data('getEnergyAmountForUser', [user.address])
print(user_energy)